In [1]:
#Step1 開始訓練偵測MB的腦區，輸出訓練資料在: 
# 使用1 GPU
!python train.py --weights weights/yolov7x_training.pt --cfg cfg/training/yolov7x_MB.yaml --data data/voc_MB_1type.yaml --epochs 1000 --batch-size 32 --img-size 640 --project mb_trained/train

YOLOR 🚀 2022-11-10 torch 1.9.0+cu111 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7x_MB.yaml', data='data/voc_MB_1type.yaml', device='', entity=None, epochs=1000, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='MB_trained/train', quad=False, rect=False, resume=False, save_dir='MB_trained/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, v5_metric=False, weights='weights/yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir MB_trained/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.000

In [12]:
# Step2 完成訓練後，推理先前人工確認的圖片，測試是否準確度夠高
# 先將需要推理的圖片複製到 人工驗證 資料夾
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import shutil
import time
import random
# 600張隨機人工驗證完成的圖片路徑: /raid/LoLab/kaiyi_hsu/DASH_Website/assets/True
# 讀取 路徑: /raid/LoLab/kaiyi_hsu/DASH_Website/assets/True/ 內的檔案名稱
human_validated_path = '/raid/LoLab/kaiyi_hsu/DASH_Website/assets/True/'
# 讀取 路徑: /raid/LoLab/kaiyi_hsu/DASH_Website/assets/True/ 內的檔案名稱
human_validated_files = os.listdir(human_validated_path)
print('人工驗證圖片 總共數量: ',len(human_validated_files))
# 讀取圖片的路徑
XY_view_path = '/raid/LoLab/kaiyi_hsu/YOLO/果蠅腦XY圖_人工標記Label/XY_View/'
# 複製 XY_view_path 路徑中 600張human_validated_files 到 /raid/LoLab/kaiyi_hsu/2023_YOLOv7_FCN/Neuron_image_and_label/人工驗證/ 資料夾
output_txt_path = '/raid/LoLab/kaiyi_hsu/2023_YOLOv7_FCN/Neuron_image_and_label/人工驗證/'
for i in range(len(human_validated_files)):
    shutil.copy(XY_view_path+human_validated_files[i], output_txt_path+human_validated_files[i])

人工驗證圖片 總共數量:  600


In [13]:
# Step3 開始推理 exp是投影Z加總的模型、exp2 是Zsum & Zmax都有訓練的模型
# GT顏色 = #FFC000 = (0, 192, 255) 問GPT: #FFC000 對應BGR表示法[detect.py 預設是GT的顏色]
# MB顏色 = #00B0F0 = (240, 176, 0) 問GPT: #00B0F0 對應BGR表示法
# AL顏色 = 
!python detect.py --conf-thres 0.5 --color 240 176 0 --weights ./mb_trained/train/exp2/weights/best.pt --source Neuron_image_and_label/人工驗證 --img-size 640 --save-txt --project mb_inference/Test

opt==== Namespace(agnostic_nms=False, augment=False, classes=None, color=[240, 176, 0], conf_thres=0.5, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='mb_inference/Test', save_conf=False, save_txt=True, source='Neuron_image_and_label/人工驗證', update=False, view_img=False, weights=['./mb_trained/train/exp2/weights/best.pt'])
custom_color:  [240, 176, 0]
YOLOR 🚀 2022-11-10 torch 1.9.0+cu111 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536.1875MB)

Fusing layers... 
IDetect.fuse
/root/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 362 layers, 70782444 parameters, 0 gradients, 188.0

In [1]:
# Step4 檢查沒有預測出MB的大腦編號(人工檢視)
import os
# 等待推理的資料夾: Neuron_image_and_label/人工驗證/
input_dir = 'Neuron_image_and_label/人工驗證/'
# YOLO推理輸出的資料夾: al_inference/Test/exp/
output_dir = 'mb_inference/Test/exp/'
# 列出所有等待推理的圖片
input_files = os.listdir(input_dir)
print('等待推理的圖片數量: ',len(input_files))
# 列出所有YOLO推理輸出的圖片，只列出副檔名為png的圖片
output_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
print('完成推理的圖片數量: ',len(output_files))
# 比對input_files與output_files，找出沒有推理輸出的圖片(紀錄在沒有推理輸出的圖片的名稱至列表，並記錄總數)
no_output_files = []
for input_file in input_files:
    if input_file not in output_files:
        no_output_files.append(input_file)
print('沒有推理輸出的圖片數量: ', len(no_output_files))

等待推理的圖片數量:  600
完成推理的圖片數量:  600
沒有推理輸出的圖片數量:  0
